In [1]:
import glob, json
import numpy as np
import pandas as pd
from scipy.stats import entropy
import sklearn.decomposition
import sklearn.manifold
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from lib import *
from pathlib import Path
plt.style.use(Path('../peptidome.mplstyle'))

/media/andreas/data/repos/peptidome/code/netmhc/../lib/main.py:419: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def mcmcsampler(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1):


In [2]:
datadir_maxent = '/home/andreas/drop/data/peptidome/maxent/'

In [3]:
proteome = 'Humanviruses'
k = 9
all_data = {}
for i, model in enumerate(['independent', 'ncov', 'nskew', 'nskewfcov']):

    arr = np.load('../classifier/data/{proteome}_{model}_k{k}_likelihoods.npz'.format(proteome=proteome,
                                                                                      model=model, k=k))
    df = pd.read_csv(datadir+'maxent/data/{proteome}_{model}_k{k}_dkl.csv'.format(proteome=proteome,
                                                                             model=model, k=k),
                     index_col=0, squeeze=True)
    dkl = df.loc['DKL']
    data = {}

    logp_hh, logp_pp, logp_hp, logp_ph = arr['logp_hh'], arr['logp_pp'], arr['logp_hp'], arr['logp_ph']
    data['logp_hh'], data['logp_pp'], data['logp_hp'], data['logp_ph'] = logp_hh, logp_pp, logp_hp, logp_ph

    positive = logp_pp-logp_ph
    negative = logp_hp-logp_hh
    data['positive'] = np.sort(positive)*np.log10(np.exp(1))
    data['negative'] = np.sort(negative)*np.log10(np.exp(1))
    data['dkl'] = dkl
    
    all_data[model] = data

FileNotFoundError: [Errno 2] No such file or directory: '../classifier/data/Humanviruses_independent_k9_likelihoods.npz'

In [4]:
human_kmers = load_matrix(datadir_maxent+'Human_test_k{k}_matrix.csv.gz'.format(k=k))
pathogen_kmers = load_matrix(datadir_maxent+'{proteome}_test_k{k}_matrix.csv.gz'.format(proteome=proteome, k=k))

In [5]:
human_kmers_str = matrix_to_kmers(human_kmers)

In [6]:
pathogen_kmers_str = matrix_to_kmers(pathogen_kmers)

In [7]:
with open('data/human.txt', 'w') as f:
    for item in human_kmers_str[:100000]:
        f.write("%s\n" % item)

In [8]:
with open('data/pathogen.txt', 'w') as f:
    for item in pathogen_kmers_str[:100000]:
        f.write("%s\n" % item)

In [9]:
from lib.netmhcrunutils import run_netMHC

In [10]:
dfhla = pd.read_csv(datadir+'hlas.csv', sep='\t', skiprows=1)
hlas = dfhla['name']
#hlas = ['HLA-A0202']

In [11]:
for hla in hlas:
    run_netMHC('data/human.txt', 'data/human', hla, binder_only=False)

/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-A2602.csv -a HLA-A2602
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-A2603.csv -a HLA-A2603
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-A2902.csv -a HLA-A2902
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-A3001.csv -a HLA-A3001
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -

/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-B4201.csv -a HLA-B4201
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-B4402.csv -a HLA-B4402
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-B4403.csv -a HLA-B4403
/media/andreas/data/repos/peptidome/dependencies/netMHC-4.0/netMHC -f /media/andreas/data/repos/peptidome/code/netmhc/data/human.txt -xls -xlsfile /media/andreas/data/repos/peptidome/code/netmhc/data/human-HLA-B4501.csv -a HLA-B4501


KeyboardInterrupt: 

In [ ]:
for hla in hlas:
    run_netMHC('data/pathogen.txt', 'data/pathogen', hla, binder_only=False)

In [ ]:
proteome = 'Humanviruses'
model = 'nskewfcov'
arr = np.load('../classifier/data/{proteome}_{model}_k{k}_likelihoods.npz'.format(proteome=proteome,
                                                                                      model=model, k=k))
prob_ratio_pathogen = (arr['logp_pp'] - arr['logp_ph'])[:100000]
prob_ratio_human = (arr['logp_hp'] - arr['logp_hh'])[:100000]
prob_pathogen = arr['logp_pp'][:100000]
prob_human = arr['logp_hh'][:100000]

In [ ]:
proteome = 'pathogen'
correlations_pathogen = []
correlations_pathogen_ratio = []
correlations_pathogen_ratio_abs = []

for hla in hlas:
    df = pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla), sep='\t', skiprows=1)
    corr, pval = scipy.stats.spearmanr(prob_pathogen, np.array(df['nM']))
    correlations_pathogen.append(corr)
    corr, pval = scipy.stats.spearmanr(prob_ratio_pathogen, np.array(df['nM']))
    correlations_pathogen_ratio.append(corr)
    corr, pval = scipy.stats.spearmanr(np.abs(prob_ratio_pathogen), np.array(df['nM']))
    correlations_pathogen_ratio_abs.append(corr)

In [ ]:
proteome = 'human'
correlations_human = []
correlations_human_ratio = []
correlations_human_ratio_abs = []
for hla in hlas:
    df = pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla), sep='\t', skiprows=1)
#    q = np.quantile(df['nM'], 0.01)
    corr, pval = scipy.stats.spearmanr(prob_human, np.array(df['nM']))
    correlations_human.append(corr)
    corr, pval = scipy.stats.spearmanr(prob_ratio_human, np.array(df['nM']))
    correlations_human_ratio.append(corr)
    corr, pval = scipy.stats.spearmanr(np.abs(prob_ratio_human), np.array(df['nM']))
    correlations_human_ratio_abs.append(corr)

In [ ]:
fig, axes = plt.subplots(figsize=(3.5, 2.5), nrows=2, ncols=2, sharex=True, sharey=True)
bins = np.linspace(-0.35, 0.35, 20)
axes[0, 0].hist(correlations_pathogen, bins=bins)
axes[0, 0].set_ylabel('Pathogen')
axes[0, 0].set_title('Likelihood')
axes[0, 1].hist(correlations_pathogen_ratio, bins=bins)
axes[0, 1].set_title('Likelihood ratio $P_P/P_H$')
#axes[0, 2].hist(correlations_pathogen_ratio_abs, bins=bins);
#axes[0, 2].set_title('Abs(Likelihood ratio)')
axes[1, 0].hist(correlations_human, bins=bins)
axes[1, 0].set_ylabel('Human')
axes[1, 1].hist(correlations_human_ratio, bins=bins)
#axes[1, 2].hist(correlations_human_ratio_abs, bins=bins);
axes[0, 0].set_xlim(min(bins), max(bins))
for i in range(2):
    axes[1, i].set_xlabel(r'$\rho$ rank affinity')
fig.savefig('netmhc_correlations.pdf')

# What fraction of all peptides binds?

In [ ]:
for hla in hlas:
    df = pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla),
                             sep='\t', skiprows=1)

    print(hla, df[df['N_binders']==1]['nM'].max())

In [ ]:
pbinding = {}
for proteome in ['human', 'pathogen']:
    pbinding[proteome] = np.array([np.mean(pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla),
                         sep='\t', skiprows=1)['nM']<500)
                              for hla in hlas])

In [ ]:
df_pbinding = pd.DataFrame(dict(hla=hlas, human=pbinding['human'], pathogen=pbinding['pathogen']))
df_pbinding.head()

In [ ]:
plt.hist(df_pbinding['ratio']);

In [ ]:
pmin = 1e-3
df_pbinding = df_pbinding[(df_pbinding['human']>pmin) & (df_pbinding['pathogen']>pmin)]
df_pbinding['ratio'] = df_pbinding['pathogen']/df_pbinding['human']

In [ ]:
df_pbinding['supertype'] = df_pbinding['hla'].apply(lambda s: s[4])

In [ ]:
bins = np.arange(0.7, 1.6, 0.1)
df_pbinding.groupby('supertype')['ratio'].hist(bins=bins, histtype='stepfilled', alpha=.5);

# Regression with likelihoods

In [ ]:
sys.path.append('/home/amayer/repos/tcr_space/code')
import tcrspacelib as tl

In [ ]:
prob_ratios = {}
prob_ratios['human'] = prob_ratio_human
prob_ratios['pathogen'] = prob_ratio_pathogen

In [ ]:
def histedges_equalN(x, nbin):
    npt = len(x)
    return np.interp(np.linspace(0, npt, nbin + 1),
                     np.arange(npt),
                     np.sort(x))

In [ ]:
probs = dict(human=prob_human, pathogen=prob_pathogen)

In [ ]:
bins = histedges_equalN(probs['pathogen'], 10)[1:-1]
for hla in ['HLA-A0202']:#hlas.sample(10):
    print(hla)
    fig, ax = plt.subplots()
    fig.suptitle(hla)
    for proteome in ['human', 'pathogen']:
        #prob = prob_ratios[proteome]
        prob = probs[proteome]

        df = pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla),
                         sep='\t', skiprows=1)
        print(proteome, np.mean(np.array(df['nM'])<500))
        bins = np.linspace(0, 1, 5)
        tl.regplot(scipy.stats.rankdata(prob)/len(prob),#prob,
                   np.array(df['nM'])<500, bins=bins, label=proteome)
    ax.set_xlabel('Likelihood rank')
    ax.set_ylabel('Fraction presented')
    ax.set_xlim(0.0)
    ax.legend()

In [ ]:
bins = histedges_equalN(probs['pathogen'], 10)[1:-1]
for hla in ['HLA-A0202']:#hlas.sample(10):
    print(hla)
    fig, ax = plt.subplots()
    fig.suptitle(hla)
    for prob, label in [(prob_pathogen, 'likelihood'), (prob_ratio_pathogen, 'likelihood ratio')]:
        df = pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla),
                         sep='\t', skiprows=1)
        print(proteome, np.mean(np.array(df['nM'])<500))
        bins = np.linspace(0, 1, 5)
        tl.regplot(scipy.stats.rankdata(prob)/len(prob),
                   np.array(df['nM'])<500, bins=bins, label=label)
    ax.set_xlabel('Likelihood rank')
    ax.set_ylabel('Fraction presented')
    ax.set_xlim(0.0)
    ax.legend()

In [ ]:
fig, ax = plt.subplots()
plot_histograms([prob_ratio_human, prob_ratio_pathogen],
                ['human', 'pathogen'],
                nbins=50, lw=2,
                xmin=-2, xmax=2, ax=ax, step=False)

In [ ]:
background = np.mean(prob_ratio_pathogen) - np.mean(prob_ratio_human)

In [ ]:
dkls = []
ratios = []
for hla in dfhla['name']:
    dfs = {}
    for proteome in ['human', 'pathogen']:
        df = pd.read_csv('data/{proteome}-{hla}.csv'.format(proteome=proteome, hla=hla), sep='\t', skiprows=1)
        dfs[proteome] = df
    ratio = np.sum(dfs['pathogen']['Rank'] < 10)/np.sum(dfs['human']['Rank'] < 10)
    ratios.append(ratio)
    dkl = np.mean(prob_ratio[dfs['pathogen']['Rank'] < 1]) - np.mean(prob_ratio_human[dfs['human']['Rank'] < 1])
    dkls.append(dkl)
    #print(hla, dkl, dp)

In [ ]:
plt.hist(dkls)
plt.axvline(background, color='k');

In [ ]:
plt.hist(np.log2(ratios))
top_ratio = all_data['nskewfcov']['positive']
top_ratio = np.mean(neg_ratio[-int(0.1*len(neg_ratio)):])/np.log(2)
plt.axvline(top_ratio)

In [ ]:
proteome = 'Humanviruses'
data = {}
for model in ['independent', 'nskewfcov']:
    arr = np.load('../classifier/data/{proteome}_{model}_k{k}_likelihoods.npz'.format(proteome=proteome,
                                                                                      model=model, k=k))
    data[model] = arr['logp_hh']

In [ ]:
density_scatter(data['independent'], data['nskewfcov'])

In [ ]:
scipy.stats.spearmanr(data['independent'], data['nskewfcov'])

In [ ]:
dfiedb = load_iedb_tcellepitopes(human_only=True)

In [ ]:
dfiedb['MHC', 'Allele Name'].value_counts().head(30)